<a href="https://colab.research.google.com/github/ExCaLBBR/ExCaLBBR_Projects/blob/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions_APF/imaging/code/PsychoPyOutput_1stLvlAnalyses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gorilla Info: <br>
Project: RaciallyBiasedDecisions_APF <br>
Account: Roberto Vargas (robertov@andrew.cmu.edu)

In [1]:
## @title Install dependancies
#Load relevant libraries and install dependencies
import pandas as pd
import numpy as np
import random
from pickle import TRUE

In [2]:
#@title Parameter Specification
FullList = {'RBDCW001', 'RBDCW002', 'RBDCW003', 'RBDCW004', 'RBDCW005', 'RBDCW006','RBDCW007', 'RBDCW008', 'RBDCW009', 'RBDCW010', 'RBDCW011'}

In [3]:
#@title Load data for all participants

iat_data = {}
# Loop through the list of participants
for Participant in FullList:
    iat_data[Participant] = {}  # Initialize a nested dictionary for each participant
    try:# Run1 data
        url_run1 = f'https://raw.githubusercontent.com/ExCaLBBR/ExCaLBBR_Projects/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions_APF/imaging/psychoPy/behOutput/raw/{Participant}_Run1_IAT_fMRI.csv'
        df_run1 = pd.read_csv(url_run1)
        iat_data[Participant]['Run1'] = df_run1
    except Exception:
        iat_data[Participant]['Run1'] = []
        print(f"IATRun1 does not exist for participant: {Participant}")


    try:# Run2 data
        url_run2 = f'https://raw.githubusercontent.com/ExCaLBBR/ExCaLBBR_Projects/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions_APF/imaging/psychoPy/behOutput/raw/{Participant}_Run2_IAT_fMRI.csv'
        df_run2 = pd.read_csv(url_run2)
        iat_data[Participant]['Run2'] = df_run2
    except Exception:
        iat_data[Participant]['Run2'] = []
        print(f"IATRun3 does not exist for participant: {Participant}")


    try:# Run3 data
        url_run3 = f'https://raw.githubusercontent.com/ExCaLBBR/ExCaLBBR_Projects/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions_APF/imaging/psychoPy/behOutput/raw/{Participant}_Run3_IAT_fMRI.csv'
        df_run3 = pd.read_csv(url_run3)
        iat_data[Participant]['Run3'] = df_run3

    except Exception:
        iat_data[Participant]['Run3'] = []
        print(f"IATRun3 does not exist for participant: {Participant}")


# Loop through the adventure
adventure_data = {}
for Participant in FullList:
    try:
        adventure_data[Participant] = {}

        # Load Run1 adventure task data
        url_advrun1 = f'https://raw.githubusercontent.com/ExCaLBBR/ExCaLBBR_Projects/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions_APF/imaging/psychoPy/behOutput/raw/{Participant}_Run1_AdvTask.csv'
        df_advrun1 = pd.read_csv(url_advrun1)
        adventure_data[Participant]['AdvRun1'] = df_advrun1

        # Print confirmation message
        #print(f"Successfully loaded and saved Run1 adventure data for participant {Participant}")

    except Exception as e:
        # Print error message if data loading fails
        print(f"Error processing adventure data1 for participant {Participant}: {e}")

    try:
        # Load Run2 adventure task data
        url_advrun2 = f'https://raw.githubusercontent.com/ExCaLBBR/ExCaLBBR_Projects/main/RaciallyBiasedDecisions/RaciallyBiasedDecisions_APF/imaging/psychoPy/behOutput/raw/{Participant}_Run2_AdvTask.csv'
        df_advrun2 = pd.read_csv(url_advrun2)
        adventure_data[Participant]['AdvRun2'] = df_advrun2

        # Print confirmation message
        #print(f"Successfully loaded and saved Run2 adventure data for participant {Participant}")

    except Exception as e:
        # Print error message if data loading fails
        print(f"Error processing adventure data2 for participant {Participant}: {e}")

IATRun3 does not exist for participant: RBDCW009
IATRun3 does not exist for participant: RBDCW001


In [5]:
#@title Generate Contrast event_file for IAT
for p in iat_data:
    try:  # Run 1 processing
        IATRun1 = iat_data[p]['Run1']
        discIndx = IATRun1['SyncPulse_Discard.started'].dropna().index
        discSync = IATRun1['SyncPulse_Discard.started'][discIndx] + IATRun1['SyncPulse_Discard.rt'][discIndx]
        scannerOffset = discSync.iloc[0]  # Account for difference between PsychPy starting and 1st sync pulse being received

        # WordsImages_SW volumes
        wordsImageSWIndx = IATRun1['ImageCentre_wisw'].dropna().index
        wordsImageSWSync = np.round(IATRun1['SyncPulse_Trial.started'][wordsImageSWIndx] + IATRun1['SyncPulse_Trial.rt'][wordsImageSWIndx] - scannerOffset, 0)
        respRT_imgOffsetSW = np.round(IATRun1['key_resp_wisw.rt'][wordsImageSWIndx], 0)
        wordsImageSW_LRkeyImg = wordsImageSWSync + respRT_imgOffsetSW  # used in final event file
        wordsImageKeyLabelSW = IATRun1['key_resp_wisw.keys'].replace({2: "Left", 1: "Right"})  # renamed the left and right
        wordsImageKeyLabelSW = [x for x in wordsImageKeyLabelSW if str(x) != 'nan']  # used in final event file
        wordImagesSW_ITI = np.round(IATRun1['ITI'][wordsImageSWIndx], 0)  # used in final event file

        # WordsImages_WS volumes
        wordsImageWSIndx = IATRun1['ImageCentre_wiws'].dropna().index
        wordsImageWSSync = np.round(IATRun1['SyncPulse_Trial.started'][wordsImageWSIndx] + IATRun1['SyncPulse_Trial.rt'][wordsImageWSIndx] - scannerOffset, 0)
        respRT_imgOffsetWS = np.round(IATRun1['key_resp_wiws.rt'][wordsImageWSIndx], 0)
        wordsImageWS_LRkeyImg = wordsImageWSSync + respRT_imgOffsetWS  # used in final event file
        wordsImageKeyLabelWS = IATRun1['key_resp_wiws.keys'].replace({2: "Left", 1: "Right"})  # renamed the left and right
        wordsImageKeyLabelWS = [x for x in wordsImageKeyLabelWS if str(x) != 'nan']  # used in final event file
        wordImagesWS_ITI = np.round(IATRun1['ITI'][wordsImageWSIndx], 0)  # used in final event file

        # Concatenation here
        label1 = wordsImageKeyLabelSW + wordsImageKeyLabelWS
        onset1 = pd.concat([wordsImageSW_LRkeyImg, wordsImageWS_LRkeyImg])
        onset1 = onset1.reset_index(drop=True)
        duration1 = pd.concat([wordImagesSW_ITI, wordImagesWS_ITI])
        duration1 = duration1.reset_index(drop=True)
        events = pd.DataFrame({'trial_type': label1, 'onset': onset1, 'duration': duration1})
        events.to_csv(p+'_eventsIATRun1.csv', sep="\t", index=False)
    except Exception as e:
        print(f"IATRun1 does not exist for participant: {p} with error {e}")

    try:  # Run 2 processing
        IATRun2 = iat_data[p]['Run2']
        discIndx2 = IATRun2['SyncPulse_Discard.started'].dropna().index
        discSync2 = IATRun2['SyncPulse_Discard.started'][discIndx2] + IATRun2['SyncPulse_Discard.rt'][discIndx2]
        scannerOffset2 = discSync2.iloc[0]

        # WordsImages_SI volumes
        wordsImageSIIndx = IATRun2['ImageCentre_wisi'].dropna().index
        wordsImageSISync = np.round(IATRun2['SyncPulse_Trial.started'][wordsImageSIIndx] + IATRun2['SyncPulse_Trial.rt'][wordsImageSIIndx] - scannerOffset2, 0)
        respRT_imgOffsetSI = np.round(IATRun2['key_resp_wisi.rt'][wordsImageSIIndx], 0)
        wordsImageSI_LRkeyImg = wordsImageSISync + respRT_imgOffsetSI  # used in final event file
        wordsImageKeyLabelSI = IATRun2['key_resp_wisi.keys'].replace({2: "Left", 1: "Right"})  # renamed the left and right
        wordsImageKeyLabelSI = [x for x in wordsImageKeyLabelSI if str(x) != 'nan']  # used in final event file
        wordImagesSI_ITI = np.round(IATRun2['ITI'][wordsImageSIIndx], 0)  # used in final event file

        # WordsImages_IS volumes
        wordsImageISIndx = IATRun2['ImageCentre_wiis'].dropna().index
        wordsImageISSync = np.round(IATRun2['SyncPulse_Trial.started'][wordsImageISIndx] + IATRun2['SyncPulse_Trial.rt'][wordsImageISIndx] - scannerOffset2, 0)
        respRT_imgOffsetIS = np.round(IATRun2['key_resp_wiis.rt'][wordsImageISIndx], 0)
        wordsImageIS_LRkeyImg = wordsImageISSync + respRT_imgOffsetIS  # used in final event file
        wordsImageKeyLabelIS = IATRun2['key_resp_wiis.keys'].replace({2: "Left", 1: "Right"})  # renamed the left and right
        wordsImageKeyLabelIS = [x for x in wordsImageKeyLabelIS if str(x) != 'nan']  # used in final event file
        wordImagesIS_ITI = np.round(IATRun2['ITI'][wordsImageISIndx], 0)  # used in final event file

        # Concatenation here
        label2 = wordsImageKeyLabelSI + wordsImageKeyLabelIS
        onset2 = pd.concat([wordsImageSI_LRkeyImg, wordsImageIS_LRkeyImg])
        onset2 = onset2.reset_index(drop=True)
        duration2 = pd.concat([wordImagesSI_ITI, wordImagesIS_ITI])
        duration2 = duration2.reset_index(drop=True)
        events = pd.DataFrame({'trial_type': label2, 'onset': onset2, 'duration': duration2})
        events.to_csv(p+'_eventsIATRun2.csv', sep="\t", index=False)
    except Exception as e:
        print(f"IATRun2 does not exist for participant: {p} with error {e}")

    try:  # Run 3 processing
        IATRun3 = iat_data[p]['Run3']
        discIndx3 = IATRun3['SyncPulse_Discard.started'].dropna().index
        discSync3 = IATRun3['SyncPulse_Discard.started'][discIndx3] + IATRun3['SyncPulse_Discard.rt'][discIndx3]
        scannerOffset3 = discSync3.iloc[0]

        # WordsImages_BG volumes
        wordsImageBGIndx = IATRun3['ImageCentre_wibg'].dropna().index
        wordsImageBGSync = np.round(IATRun3['SyncPulse_Trial.started'][wordsImageBGIndx] + IATRun3['SyncPulse_Trial.rt'][wordsImageBGIndx] - scannerOffset3, 0)
        respRT_imgOffsetBG = np.round(IATRun3['key_resp_wibg.rt'][wordsImageBGIndx], 0)
        wordsImageBG_LRkeyImg = wordsImageBGSync + respRT_imgOffsetBG  # used in final event file
        wordsImageKeyLabelBG = IATRun3['key_resp_wibg.keys'].replace({2: "Left", 1: "Right"})  # renamed the left and right
        wordsImageKeyLabelBG = [x for x in wordsImageKeyLabelBG if str(x) != 'nan']  # used in final event file
        wordImagesBG_ITI = np.round(IATRun3['ITI'][wordsImageBGIndx], 0)  # used in final event file

        # WordsImages_GB volumes
        wordsImageGBIndx = IATRun3['ImageCentre_wigb'].dropna().index
        wordsImageGBSync = np.round(IATRun3['SyncPulse_Trial.started'][wordsImageGBIndx] + IATRun3['SyncPulse_Trial.rt'][wordsImageGBIndx] - scannerOffset3, 0)
        respRT_imgOffsetGB = np.round(IATRun3['key_resp_wigb.rt'][wordsImageGBIndx], 0)
        wordsImageGB_LRkeyImg = wordsImageGBSync + respRT_imgOffsetGB  # used in final event file
        wordsImageKeyLabelGB = IATRun3['key_resp_wigb.keys'].replace({2: "Left", 1: "Right"})  # renamed the left and right
        wordsImageKeyLabelGB = [x for x in wordsImageKeyLabelGB if str(x) != 'nan']  # used in final event file
        wordImagesGB_ITI = np.round(IATRun3['ITI'][wordsImageGBIndx], 0)  # used in final event file

        # Concatenation here
        label3 = wordsImageKeyLabelBG + wordsImageKeyLabelGB
        onset3 = pd.concat([wordsImageBG_LRkeyImg, wordsImageGB_LRkeyImg])
        onset3 = onset3.reset_index(drop=True)
        duration3 = pd.concat([wordImagesBG_ITI, wordImagesGB_ITI])
        duration3 = duration3.reset_index(drop=True)
        events = pd.DataFrame({'trial_type': label3, 'onset': onset3, 'duration': duration3})
        events.to_csv(p+'_eventsIATRun3.csv', sep="\t", index=False)
    except Exception as e:
        print(f"IATRun3 does not exist for participant: {p} with error {e}")



IATRun3 does not exist for participant: RBDCW009 with error list indices must be integers or slices, not str
IATRun3 does not exist for participant: RBDCW001 with error list indices must be integers or slices, not str


In [128]:
#@title Generate Contrast event_file for Adventure Task
for p in adventure_data:
    # try:  # Run 1 processing
    #     ADVRun1 = adventure_data[p]['AdvRun1']
    #     discIndx = ADVRun1['SyncPulse_discard.started'].dropna().index
    #     discSync = ADVRun1['SyncPulse_discard.started'][discIndx] + ADVRun1['SyncPulse_discard.rt'][discIndx]
    #     scannerOffset_adv1 = discSync.iloc[0]  # Account for difference between PsychPy starting and 1st sync pulse being received

    #     # Encounter trial volumes
    #     EncIndx = ADVRun1['TrialNum'].dropna().index
    #     EncSync = np.round(ADVRun1['SyncPulse_Action.started'][EncIndx] + ADVRun1['SyncPulse_Action.rt'][EncIndx] - scannerOffset_adv1, 0)

    #     #Extract RTs
    #     rtPun = ADVRun1['player_action_keys_punish.rt'][EncIndx].dropna()
    #     rtRew = ADVRun1['player_action_keys_reward.rt'][EncIndx].dropna()
    #     rtAction = pd.concat([rtPun,rtRew], names='ActionRT')
    #     rtAction = rtAction.sort_index(ascending=True)
    #     rtAction = rtAction.rename('rtAction')
    #     rtAction = rtAction.reset_index()

    #     #Extract LR key presses
    #     punIndx = ADVRun1['player_action_keys_punish.keys'][EncIndx].dropna().index
    #     ADVRun1.loc[punIndx,'player_action_keys_punish.keys'] = 'Punish'
    #     rewIndx = ADVRun1['player_action_keys_reward.keys'][EncIndx].dropna().index
    #     ADVRun1.loc[rewIndx,'player_action_keys_reward.keys'] = 'Reward'
    #     rewPunAction = pd.concat([ADVRun1['player_action_keys_punish.keys'][punIndx],ADVRun1['player_action_keys_reward.keys'][rewIndx]])
    #     rewPunAction = rewPunAction.sort_index(ascending=True)
    #     rewPunAction = rewPunAction.rename('labAction')
    #     rewPunAction = rewPunAction.reset_index()

    #     #Check for changes in responses
    #     idx = pd.Index(rtAction['index'])
    #     dupidx = idx.duplicated(keep = False)
    #     dupCases = rtAction[dupidx] #Refer to 0 dataframe

    #  if not dupCases.empty:
    #    lengths = []
    #    for ix in dupCases['rtAction']:
    #        lengths.append(len(ix)) #
    #    dupCases.insert(2, "Len", lengths, True)
    #    dropIdx = dupCases[['Len']].idxmin()

    #    # Drop duplicates
    #    rewPunAction = rewPunAction2drop(dropIdx) # used in final event file label
    #    rtAction = rtAction.drop(dropIdx2)

    #     #Convert rt strings to numbers
    #     respRT_imgOffset = [float(x[1:16]) for x in rtAction['rtAction']]
    #     respRT_imgOffset = np.round(respRT_imgOffset,0)
    #     EncAction_LRkeyImg = EncSync + respRT_imgOffset  # used in final event file imgage number

    #     #Extract ITI
    #     trialITI = ADVRun1['scenDelay'][EncIndx]  # used in final event file ITI
    #     trialITI = trialITI.reset_index(drop=True)

    #     #Concatenation here
    #     label1 =rewPunAction
    #     label1 = label1.reset_index(drop=True)
    #     onset1 = EncAction_LRkeyImg
    #     onset1 = onset1.reset_index(drop=True)
    #     duration1 = trialITI
    #     events = pd.DataFrame({'trial_type': label1['labAction'], 'onset': onset1, 'duration': duration1})
    #     events.to_csv(p+'_eventsIATRun3.csv', sep="\t", index=False)
    # except Exception as e:
    #     print(f"ADVRun1 does not exist for participant: {p} with error {e}")

    try:  # Run 2 processing
    # Adventure Task 2
        ADVRun2 = adventure_data[p]['AdvRun2']
        discIndx2 = ADVRun2['SyncPulse_discard.started'].dropna().index
        discSync2 = ADVRun2['SyncPulse_discard.started'][discIndx2] + ADVRun2['SyncPulse_discard.rt'][discIndx2]
        scannerOffset_adv2 = discSync2.iloc[0]  # Account for difference between PsychPy starting and 1st sync pulse being received

         # Encounter trial volumes
        EncIndx2 = ADVRun2['TrialNum'].dropna().index
        EncSync2 = np.round(ADVRun2['SyncPulse_Action.started'][EncIndx2] + ADVRun2['SyncPulse_Action.rt'][EncIndx2] - scannerOffset_adv2, 0)

        # Extract RTs
        rtPun2 = ADVRun2['player_action_keys_punish.rt'][EncIndx2].dropna()
        rtRew2 = ADVRun2['player_action_keys_reward.rt'][EncIndx2].dropna()
        rtAction2 = pd.concat([rtPun2,rtRew2], names='ActionRT')
        rtAction2 = rtAction2.sort_index(ascending=True)
        rtAction2 = rtAction2.rename('rtAction')
        rtAction2 = rtAction2.reset_index()

        # Extract LR key presses
        punIndx2 = ADVRun2['player_action_keys_punish.keys'][EncIndx2].dropna().index
        ADVRun2.loc[punIndx2,'player_action_keys_punish.keys'] = 'Punish'
        rewIndx2 = ADVRun2['player_action_keys_reward.keys'][EncIndx2].dropna().index
        ADVRun2.loc[rewIndx2,'player_action_keys_reward.keys'] = 'Reward'
        rewPunAction2 = pd.concat([ADVRun2['player_action_keys_punish.keys'][punIndx2],ADVRun2['player_action_keys_reward.keys'][rewIndx2]])
        rewPunAction2 = rewPunAction2.sort_index(ascending=True)
        rewPunAction2 = rewPunAction2.rename('labAction')
        rewPunAction2 = rewPunAction2.reset_index()

        # Check for changes in responses
        idx2 = pd.Index(rtAction2['index'])
        dupidx2 = idx2.duplicated(keep = False)
        dupCases2 = rtAction2[dupidx2] #Refer to 0 dataframe

        if not dupCases2.empty:
          lengths2 = []
          for ix in dupCases2['rtAction']:
              lengths2.append(len(ix)) #
          dupCases2.insert(2, "Len", lengths2, True)
          dropIdx2 = dupCases2[['Len']].idxmin()

          # Drop duplicates
          rewPunAction2 = rewPunAction2.drop(dropIdx2) # used in final event file label
          rtAction2 = rtAction2.drop(dropIdx2)

        # Convert rt strings to numbers
        respRT_imgOffset2 = [float(x[1:16]) for x in rtAction2['rtAction']]
        respRT_imgOffset2 = np.round(respRT_imgOffset2,0)
        EncAction_LRkeyImg2 = EncSync2 + respRT_imgOffset2  # used in final event file imgage number

        # Extract ITI
        trialITI2 = ADVRun2['scenDelay'][EncIndx2]  # used in final event file ITI
        trialITI2 = trialITI2.reset_index(drop=True)

        # Concatenation here
        label2 =rewPunAction2
        label2 = label2.reset_index(drop=True)
        onset2 = EncAction_LRkeyImg2
        onset2 = onset2.reset_index(drop=True)
        duration2 = trialITI2
        events2 = pd.DataFrame({'trial_type': label2['labAction'], 'onset': onset2, 'duration': duration2})
        events2.to_csv(p+'_eventsAdvRun2.csv', sep="\t", index=False)

    except Exception as e:
        print(f"ADVRun2 does not exist for participant: {p} with error {e}")



ADVRun2 does not exist for participant: RBDCW011 with error operands could not be broadcast together with shapes (18,) (19,) 
ADVRun2 does not exist for participant: RBDCW007 with error operands could not be broadcast together with shapes (16,) (13,) 


In [31]:
################HANDLE CASES WHERE THE PARTICIPANT DIDNT RESPOND

True

In [129]:
p = 'RBDCW007'
ADVRun2 = adventure_data[p]['AdvRun2']
discIndx2 = ADVRun2['SyncPulse_discard.started'].dropna().index
discSync2 = ADVRun2['SyncPulse_discard.started'][discIndx2] + ADVRun2['SyncPulse_discard.rt'][discIndx2]
scannerOffset_adv2 = discSync2.iloc[0]  # Account for difference between PsychPy starting and 1st sync pulse being received

EncIndx2 = ADVRun2['TrialNum'].dropna().index
EncSync2 = np.round(ADVRun2['SyncPulse_Action.started'][EncIndx2] + ADVRun2['SyncPulse_Action.rt'][EncIndx2] - scannerOffset_adv2, 0)

rtPun2 = ADVRun2['player_action_keys_punish.rt'][EncIndx2].dropna()
rtRew2 = ADVRun2['player_action_keys_reward.rt'][EncIndx2].dropna()
rtAction2 = pd.concat([rtPun2,rtRew2], names='ActionRT')
rtAction2 = rtAction2.sort_index(ascending=True)
rtAction2 = rtAction2.rename('rtAction')
rtAction2 = rtAction2.reset_index()

idx2 = pd.Index(rtAction2['index'])
dupidx2 = idx2.duplicated(keep = False)
dupCases2 = rtAction2[dupidx2] #Refer to 0 dataframe


if not dupCases2.empty:
  lengths2 = []
  for ix in dupCases2['rtAction']:
      lengths2.append(len(ix)) #
  dupCases2.insert(2, "Len", lengths2, True)
  dropIdx2 = dupCases2[['Len']].idxmin()

  # Drop duplicates
  rewPunAction2 = rewPunAction2.drop(dropIdx2) # used in final event file label
  rtAction2 = rtAction2.drop(dropIdx2)

# Convert rt strings to numbers
respRT_imgOffset2 = [float(x[1:16]) for x in rtAction2['rtAction']]
respRT_imgOffset2 = np.round(respRT_imgOffset2,0)
EncAction_LRkeyImg2 = EncSync2 + respRT_imgOffset2  # used in final event file imgage number

# Extract ITI
trialITI2 = ADVRun2['scenDelay'][EncIndx2]  # used in final event file ITI
trialITI2 = trialITI2.reset_index(drop=True)

# Concatenation here
label2 =rewPunAction2
label2 = label2.reset_index(drop=True)
onset2 = EncAction_LRkeyImg2
onset2 = onset2.reset_index(drop=True)
duration2 = trialITI2

ValueError: operands could not be broadcast together with shapes (16,) (13,) 

In [ ]:
rtAction2

In [91]:
len(ix)

56

In [79]:
lengths2 = []
lengths2

[]

In [53]:
dupCases2['rtAction'][13]

'[4.799916200000098]'

In [101]:
lengths2 = []
lengths2.append(len(dupCases2['rtAction'][13]))
lengths2.append(len(dupCases2['rtAction'][14]))

In [67]:
lengths2

[19, 56]

In [ ]:
#This is a scratch cell for testing a single iteration of the loop for the adv task
p = 'RBDCW001'
ADVRun1 = adventure_data[p]['AdvRun1']
discIndx = ADVRun1['SyncPulse_discard.started'].dropna().index
discSync = ADVRun1['SyncPulse_discard.started'][discIndx] + ADVRun1['SyncPulse_discard.rt'][discIndx]
scannerOffset_adv1 = discSync.iloc[0]  # Account for difference between PsychPy starting and 1st sync pulse being received

# Encounter trial volumes
EncIndx = ADVRun1['TrialNum'].dropna().index
EncSync = np.round(ADVRun1['SyncPulse_Action.started'][EncIndx] + ADVRun1['SyncPulse_Action.rt'][EncIndx] - scannerOffset_adv1, 0)

#Extract RTs
rtPun = ADVRun1['player_action_keys_punish.rt'][EncIndx].dropna()
rtRew = ADVRun1['player_action_keys_reward.rt'][EncIndx].dropna()
rtAction = pd.concat([rtPun,rtRew], names='ActionRT')
rtAction = rtAction.sort_index(ascending=True)
rtAction = rtAction.rename('rtAction')
rtAction = rtAction.reset_index()

#Extract LR key presses
punIndx = ADVRun1['player_action_keys_punish.keys'][EncIndx].dropna().index
ADVRun1.loc[punIndx,'player_action_keys_punish.keys'] = 'Punish'
rewIndx = ADVRun1['player_action_keys_reward.keys'][EncIndx].dropna().index
ADVRun1.loc[rewIndx,'player_action_keys_reward.keys'] = 'Reward'
rewPunAction = pd.concat([ADVRun1['player_action_keys_punish.keys'][punIndx],ADVRun1['player_action_keys_reward.keys'][rewIndx]])
rewPunAction = rewPunAction.sort_index(ascending=True)
rewPunAction = rewPunAction.rename('labAction')
rewPunAction = rewPunAction.reset_index()

#Check for changes in responses
idx = pd.Index(rtAction['index'])
dupidx = idx.duplicated(keep = False)
dupCases = rtAction[dupidx] #Refer to 0 dataframe

lengths = []
for ix in dupCases['rtAction']:
    length = lengths.append(len(ix))
dupCases.insert(2, "Len", lengths, True)
dropIdx = dupCases[['Len']].idxmin()

#Drop duplicates
rewPunAction = rewPunAction.drop(dropIdx)
rtAction = rtAction.drop(dropIdx)

#Convert rt strings to numbers
respRT_imgOffset = [float(x[1:16]) for x in rtAction['rtAction']]
respRT_imgOffset = np.round(respRT_imgOffset,0)
EncAction_LRkeyImg = EncSync + respRT_imgOffset  # used in final event file imgage number

#Extract ITI
trialITI = ADVRun1['scenDelay'][EncIndx]  # used in final event file ITI
trialITI = trialITI.reset_index(drop=True)

#Concatenation here
label1 =rewPunAction
label1 = label1.reset_index(drop=True)
onset1 = EncAction_LRkeyImg
onset1 = onset1.reset_index(drop=True)
duration1 = trialITI
events = pd.DataFrame({'trial_type': label1['labAction'], 'onset': onset1, 'duration': duration1})

In [ ]:
#@title Define desired contrast information: [Left/Right button responses]
#Desired context: Left-Right
#ToDo: Get all instances of left and right button presses and respective image stamps
# IATRun1['key_resp_wisw.keys'][wordsImageSWIndx]
# IATRun1['key_resp_wisw.rt'][wordsImageSWIndx] # the key reaction time

#IAT Run 1
#WordsImages_SW volumes
wordsImageSWIndx = IATRun1['ImageCentre_wisw'].dropna().index
wordsImageSWSync = np.round(IATRun1['SyncPulse_Trial.started'][wordsImageSWIndx]  + IATRun1['SyncPulse_Trial.rt'][wordsImageSWIndx] - scannerOffset,0)
respRT_imgOffsetSW = np.round(IATRun1['key_resp_wisw.rt'][wordsImageSWIndx],0)
wordsImageSW_LRkeyImg = wordsImageSWSync + respRT_imgOffsetSW #used in final event file
wordsImageKeyLabelSW = IATRun1['key_resp_wisw.keys'].replace({2: "Left", 1: "Right"}) # renamed the left and right
wordsImageKeyLabelSW = [x for x in wordsImageKeyLabelSW if str(x) != 'nan'] #used in final event file
wordImagesSW_ITI = np.round(IATRun1['ITI'][wordsImageSWIndx],0) #used in final event file

#WordsImages_WS volumes
wordsImageWSIndx = IATRun1['ImageCentre_wiws'].dropna().index
wordsImageWSSync = np.round(IATRun1['SyncPulse_Trial.started'][wordsImageWSIndx]  + IATRun1['SyncPulse_Trial.rt'][wordsImageWSIndx] - scannerOffset,0)
respRT_imgOffsetWS = np.round(IATRun1['key_resp_wiws.rt'][wordsImageWSIndx],0)
wordsImageWS_LRkeyImg = wordsImageWSSync + respRT_imgOffsetWS #used in final event file
wordsImageKeyLabelWS = IATRun1['key_resp_wiws.keys'].replace({2: "Left", 1: "Right"}) # renamed the left and right
wordsImageKeyLabelWS = [x for x in wordsImageKeyLabelWS if str(x) != 'nan'] #used in final event file
wordImagesWS_ITI = np.round(IATRun1['ITI'][wordsImageWSIndx],0) #used in final event file

#IAT Run 2
#WordsImages_SI volumes
wordsImageSIIndx = IATRun2['ImageCentre_wisi'].dropna().index
wordsImageSISync = np.round(IATRun2['SyncPulse_Trial.started'][wordsImageSIIndx]  + IATRun2['SyncPulse_Trial.rt'][wordsImageSIIndx] - scannerOffset2,0)
respRT_imgOffsetSI = np.round(IATRun2['key_resp_wisi.rt'][wordsImageSIIndx],0)
wordsImageSI_LRkeyImg = wordsImageSISync + respRT_imgOffsetSI #used in final event file
wordsImageKeyLabelSI = IATRun2['key_resp_wisi.keys'].replace({2: "Left", 1: "Right"}) # renamed the left and right
wordsImageKeyLabelSI = [x for x in wordsImageKeyLabelSI if str(x) != 'nan'] #used in final event file
wordImagesSI_ITI = np.round(IATRun2['ITI'][wordsImageSIIndx],0) #used in final event file

#WordsImages_IS volumes
wordsImageISIndx = IATRun2['ImageCentre_wiis'].dropna().index
wordsImageISSync = np.round(IATRun2['SyncPulse_Trial.started'][wordsImageISIndx]  + IATRun2['SyncPulse_Trial.rt'][wordsImageISIndx] - scannerOffset2,0)
respRT_imgOffsetIS = np.round(IATRun2['key_resp_wiis.rt'][wordsImageISIndx],0)
wordsImageIS_LRkeyImg = wordsImageISSync + respRT_imgOffsetIS #used in final event file
wordsImageKeyLabelIS = IATRun2['key_resp_wiis.keys'].replace({2: "Left", 1: "Right"}) # renamed the left and right
wordsImageKeyLabelIS = [x for x in wordsImageKeyLabelIS if str(x) != 'nan'] #used in final event file
wordImagesIS_ITI = np.round(IATRun2['ITI'][wordsImageISIndx],0) #used in final event file

#IAT Run 3
#WordsImages_BG volumes
wordsImageBGIndx = IATRun3['ImageCentre_wibg'].dropna().index
wordsImageBGSync = np.round(IATRun3['SyncPulse_Trial.started'][wordsImageBGIndx]  + IATRun3['SyncPulse_Trial.rt'][wordsImageBGIndx] - scannerOffset3,0)
respRT_imgOffsetBG = np.round(IATRun3['key_resp_wibg.rt'][wordsImageBGIndx],0)
wordsImageBG_LRkeyImg = wordsImageBGSync + respRT_imgOffsetBG #used in final event file
wordsImageKeyLabelBG = IATRun3['key_resp_wibg.keys'].replace({2: "Left", 1: "Right"}) # renamed the left and right
wordsImageKeyLabelBG = [x for x in wordsImageKeyLabelBG if str(x) != 'nan'] #used in final event file
wordImagesBG_ITI = np.round(IATRun3['ITI'][wordsImageBGIndx],0) #used in final event file

#WordsImages_GB volumes
wordsImageGBIndx = IATRun3['ImageCentre_wigb'].dropna().index
wordsImageGBSync = np.round(IATRun3['SyncPulse_Trial.started'][wordsImageGBIndx]  + IATRun3['SyncPulse_Trial.rt'][wordsImageGBIndx] - scannerOffset3,0)
respRT_imgOffsetGB = np.round(IATRun3['key_resp_wigb.rt'][wordsImageGBIndx],0)
wordsImageGB_LRkeyImg = wordsImageGBSync + respRT_imgOffsetGB #used in final event file
wordsImageKeyLabelGB = IATRun3['key_resp_wigb.keys'].replace({2: "Left", 1: "Right"}) # renamed the left and right
wordsImageKeyLabelGB = [x for x in wordsImageKeyLabelGB if str(x) != 'nan'] #used in final event file
wordImagesGB_ITI = np.round(IATRun3['ITI'][wordsImageGBIndx],0) #used in final event file

#Add concatonation here:
#IAT Run 1
label1 = wordsImageKeyLabelSW + wordsImageKeyLabelWS
onset1 = pd.concat([wordsImageSW_LRkeyImg, wordsImageWS_LRkeyImg])
onset1 = onset1.reset_index(drop = TRUE)
duration1 = pd.concat([wordImagesSW_ITI, wordImagesWS_ITI])
duration1 = duration1.reset_index(drop = TRUE)
events = pd.DataFrame({'trial_type':label1, 'onset':onset1, 'duration':duration1})
events.to_csv('eventsIATRun1.csv', sep = "\t")

#IAT Run 2
label2 = wordsImageKeyLabelSI + wordsImageKeyLabelIS
onset2 = pd.concat([wordsImageSI_LRkeyImg, wordsImageIS_LRkeyImg])
onset2 = onset2.reset_index(drop = TRUE)
duration2 = pd.concat([wordImagesSI_ITI, wordImagesIS_ITI])
duration2 = duration2.reset_index(drop = TRUE)
events = pd.DataFrame({'trial_type':label2, 'onset':onset2, 'duration':duration2})
events.to_csv('eventsIATRun2.csv', sep = "\t")

#IAT Run 3
label3 = wordsImageKeyLabelBG + wordsImageKeyLabelGB
onset3 = pd.concat([wordsImageBG_LRkeyImg, wordsImageGB_LRkeyImg])
onset3 = onset3.reset_index(drop = TRUE)
duration3 = pd.concat([wordImagesBG_ITI, wordImagesGB_ITI])
duration3 = duration3.reset_index(drop = TRUE)
events = pd.DataFrame({'trial_type':label3, 'onset':onset3, 'duration':duration3})
events.to_csv('eventsIATRun3.csv', sep = "\t")

In [ ]:
conditions
onsets = [list(events[events.trial_type == cond].onset) for cond in conditions]
durations = [list(events[events.trial_type == cond].duration) for cond in conditions]

In [ ]:
#@title Concatonating SyncPulse Labels and corresponding image assignments
imgTiming = pd.DataFrame({'SyncPulseID':[], 'ImgNumber':[]})

imgTiming['ImgNumber'] = pd.concat([pd.DataFrame(discSync), pd.DataFrame(baselineSync), pd.DataFrame(justImgSync), pd.DataFrame(justWordSWSync),
                                    pd.DataFrame(wordsImageSWSync), pd.DataFrame(justWordWSSync), pd.DataFrame(wordsImageWSSync)], ignore_index=True)
imgTiming['ImgNumber'] = imgTiming['ImgNumber'].round(0)

imgTiming['SyncPulseID'] = pd.concat([pd.DataFrame(['Discard']*len(discSync)), pd.DataFrame(['Baseline']*len(baselineSync)),
                                      pd.DataFrame(['Just Image']*len(justImgSync)), pd.DataFrame(['Just Word SW']*len(justWordSWSync)), pd.DataFrame(['Word Image SW']*len(wordsImageSWSync)),
                                      pd.DataFrame(['Just Word WS']*len(justWordWSSync)), pd.DataFrame(['Word Image WS']*len(wordsImageWSSync))], ignore_index = True)




In [ ]:
#@title junk attempts
#@title Define Contrast Analyses

#Loop through each participants and each run within participants
#for p in participant_data:
#  try: #Run 1 processing
#    IATRun1 = participant_data[p]['Run1']  # Initialize a nested dictionary for each participant
#    discIndx = IATRun1['SyncPulse_Discard.started'].dropna().index
#    discSync = IATRun1['SyncPulse_Discard.started'][discIndx]  + IATRun1['SyncPulse_Discard.rt'][discIndx]
#    scannerOffset = discSync[0] #Account for difference between PsychPy starting and 1st sync pulse being recieved

    #WordsImages_SW volumes
#    wordsImageSWIndx = IATRun1['ImageCentre_wisw'].dropna().index
#    wordsImageSWSync = np.round(IATRun1['SyncPulse_Trial.started'][wordsImageSWIndx]  + IATRun1['SyncPulse_Trial.rt'][wordsImageSWIndx] - scannerOffset,0)
#    respRT_imgOffsetSW = np.round(IATRun1['key_resp_wisw.rt'][wordsImageSWIndx],0)
#    wordsImageSW_LRkeyImg = wordsImageSWSync + respRT_imgOffsetSW #used in final event file
#    wordsImageKeyLabelSW = IATRun1['key_resp_wisw.keys'].replace({2: "Left", 1: "Right"}) # renamed the left and right
#    wordsImageKeyLabelSW = [x for x in wordsImageKeyLabelSW if str(x) != 'nan'] #used in final event file
#    wordImagesSW_ITI = np.round(IATRun1['ITI'][wordsImageSWIndx],0) #used in final event file

    #WordsImages_WS volumes
#    wordsImageWSIndx = IATRun1['ImageCentre_wiws'].dropna().index
#    wordsImageWSSync = np.round(IATRun1['SyncPulse_Trial.started'][wordsImageWSIndx]  + IATRun1['SyncPulse_Trial.rt'][wordsImageWSIndx] - scannerOffset,0)
#    respRT_imgOffsetWS = np.round(IATRun1['key_resp_wiws.rt'][wordsImageWSIndx],0)
#    wordsImageWS_LRkeyImg = wordsImageWSSync + respRT_imgOffsetWS #used in final event file
#    wordsImageKeyLabelWS = IATRun1['key_resp_wiws.keys'].replace({2: "Left", 1: "Right"}) # renamed the left and right
#    wordsImageKeyLabelWS = [x for x in wordsImageKeyLabelWS if str(x) != 'nan'] #used in final event file
#    wordImagesWS_ITI = np.round(IATRun1['ITI'][wordsImageWSIndx],0) #used in final event file

    #Add concatonation here:
#    label1 = wordsImageKeyLabelSW + wordsImageKeyLabelWS
#    onset1 = pd.concat([wordsImageSW_LRkeyImg, wordsImageWS_LRkeyImg])
#    onset1 = onset1.reset_index(drop = TRUE)
#    duration1 = pd.concat([wordImagesSW_ITI, wordImagesWS_ITI])
#    duration1 = duration1.reset_index(drop = TRUE)
#    events = pd.DataFrame({'trial_type':label1, 'onset':onset1, 'duration':duration1})
#    events.to_csv('eventsIATRun1.csv', sep = "\t")
#  except Exception:
#    print(f"IATRun1 does not exist for participant: {p}")

#  try: #Run 2 processing
#    IATRun2 = participant_data[p]['Run2']  # Initialize a nested dictionary for each participant
#    discIndx2 = IATRun2['SyncPulse_Discard.started'].dropna().index
#    discSync2 = IATRun2['SyncPulse_Discard.started'][discIndx2]  + IATRun2['SyncPulse_Discard.rt'][discIndx2]

    #WordsImages_SI volumes
#    wordsImageSIIndx = IATRun2['ImageCentre_wisi'].dropna().index
#    wordsImageSISync = np.round(IATRun2['SyncPulse_Trial.started'][wordsImageSIIndx]  + IATRun2['SyncPulse_Trial.rt'][wordsImageSIIndx] - scannerOffset2,0)
#    respRT_imgOffsetSI = np.round(IATRun2['key_resp_wisi.rt'][wordsImageSIIndx],0)
#    wordsImageSI_LRkeyImg = wordsImageSISync + respRT_imgOffsetSI #used in final event file
#    wordsImageKeyLabelSI = IATRun2['key_resp_wisi.keys'].replace({2: "Left", 1: "Right"}) # renamed the left and right
#    wordsImageKeyLabelSI = [x for x in wordsImageKeyLabelSI if str(x) != 'nan'] #used in final event file
#    wordImagesSI_ITI = np.round(IATRun2['ITI'][wordsImageSIIndx],0) #used in final event file

    #WordsImages_IS volumes
#    wordsImageISIndx = IATRun2['ImageCentre_wiis'].dropna().index
#    wordsImageISSync = np.round(IATRun2['SyncPulse_Trial.started'][wordsImageISIndx]  + IATRun2['SyncPulse_Trial.rt'][wordsImageISIndx] - scannerOffset2,0)
#    respRT_imgOffsetIS = np.round(IATRun2['key_resp_wiis.rt'][wordsImageISIndx],0)
#    wordsImageIS_LRkeyImg = wordsImageISSync + respRT_imgOffsetIS #used in final event file
#    wordsImageKeyLabelIS = IATRun2['key_resp_wiis.keys'].replace({2: "Left", 1: "Right"}) # renamed the left and right
#    wordsImageKeyLabelIS = [x for x in wordsImageKeyLabelIS if str(x) != 'nan'] #used in final event file
#    wordImagesIS_ITI = np.round(IATRun2['ITI'][wordsImageISIndx],0) #used in final event file

    #IAT Run 2
#    label2 = wordsImageKeyLabelSI + wordsImageKeyLabelIS
#    onset2 = pd.concat([wordsImageSI_LRkeyImg, wordsImageIS_LRkeyImg])
#   onset2 = onset2.reset_index(drop = TRUE)
#    duration2 = pd.concat([wordImagesSI_ITI, wordImagesIS_ITI])
#    duration2 = duration2.reset_index(drop = TRUE)
#    events = pd.DataFrame({'trial_type':label2, 'onset':onset2, 'duration':duration2})
#    events.to_csv('eventsIATRun2.csv', sep = "\t")
#  except Exception:
#    print(f"IATRun1 does not exist for participant: {p}")

    #IAT Run 3
    #Discarded volumes
#    IATRun3 = participant_data[p]['Run3']  # Initialize a nested dictionary for each participant
#    discIndx3 = IATRun3['SyncPulse_Discard.started'].dropna().index
#    discSync3 = IATRun3['SyncPulse_Discard.started'][discIndx3]  + IATRun3['SyncPulse_Discard.rt'][discIndx3]
#    scannerOffset3 = discSync3[0] #Account for difference between PsychPy starting and 1st sync pulse being recieved

    #WordsImages_BG volumes
 #   wordsImageBGIndx = IATRun3['ImageCentre_wibg'].dropna().index
 #   wordsImageBGSync = np.round(IATRun3['SyncPulse_Trial.started'][wordsImageBGIndx]  + IATRun3['SyncPulse_Trial.rt'][wordsImageBGIndx] - scannerOffset3,0)
 #   respRT_imgOffsetBG = np.round(IATRun3['key_resp_wibg.rt'][wordsImageBGIndx],0)
 #   wordsImageBG_LRkeyImg = wordsImageBGSync + respRT_imgOffsetBG #used in final event file
 #   wordsImageKeyLabelBG = IATRun3['key_resp_wibg.keys'].replace({2: "Left", 1: "Right"}) # renamed the left and right
 #   wordsImageKeyLabelBG = [x for x in wordsImageKeyLabelBG if str(x) != 'nan'] #used in final event file
 #   wordImagesBG_ITI = np.round(IATRun3['ITI'][wordsImageBGIndx],0) #used in final event file

    #WordsImages_GB volumes
 #   wordsImageGBIndx = IATRun3['ImageCentre_wigb'].dropna().index
 #   wordsImageGBSync = np.round(IATRun3['SyncPulse_Trial.started'][wordsImageGBIndx]  + IATRun3['SyncPulse_Trial.rt'][wordsImageGBIndx] - scannerOffset3,0)
 #   respRT_imgOffsetGB = np.round(IATRun3['key_resp_wigb.rt'][wordsImageGBIndx],0)
 #   wordsImageGB_LRkeyImg = wordsImageGBSync + respRT_imgOffsetGB #used in final event file
 #   wordsImageKeyLabelGB = IATRun3['key_resp_wigb.keys'].replace({2: "Left", 1: "Right"}) # renamed the left and right
#  wordsImageKeyLabelGB = [x for x in wordsImageKeyLabelGB if str(x) != 'nan'] #used in final event file
#    wordImagesGB_ITI = np.round(IATRun3['ITI'][wordsImageGBIndx],0) #used in final event file

#    label3 = wordsImageKeyLabelBG + wordsImageKeyLabelGB
#    onset3 = pd.concat([wordsImageBG_LRkeyImg, wordsImageGB_LRkeyImg])
#    onset3 = onset3.reset_index(drop = TRUE)
#    duration3 = pd.concat([wordImagesBG_ITI, wordImagesGB_ITI])
#    duration3 = duration3.reset_index(drop = TRUE)
#    events = pd.DataFrame({'trial_type':label3, 'onset':onset3, 'duration':duration3})
#    events.to_csv('eventsIATRun3.csv', sep = "\t")
#  except Exception:
#    print(f"IATRun1 does not exist for participant: {p}")

#reload = pd.read_csv('testevents.csv', sep="\t")
